# QAS Demo

This is a set of demo notebooks to illustrate the use of the MLTE library and SDMT process, using Quality Attribute Scenarios as guidance for the required Properties and Conditions.

NOTE: this demo has an additional set of requirements than MLTE. You can install them with the command: 

`poetry install --with demo`

If running on macOS, also install:

`poetry install --with demo-mac`


## 0. Quality Attribute Scenarios

The following are the QASs that we want to validate through the use of MLTE. The examples below relate to a hypothetical system used by visitors to a botanical garden to identify flowers in the different gardens and learn more about them. The system used an ML model that was trained on the flower category dataset [Nilsback 2008] (https://www.robots.ox.ac.uk/~vgg/data/flowers/102/). 

* **Fairness - Model Impartial to Photo Location**
  * The model receives a picture taken at the garden and, regardless of the garden location, can correctly identify the correct flowers at least 90% of the time. Test data needs to include pictures of the flowers from the different gardens, grouped by the garden that the image was taken at. The quantity of the flower images should be representative of the garden population they are taken from. The total accuracy of the model across each garden population should be higher or equal to 0.9.
* **Robustness- Model Robust to Noise (Image Blur)**
  * The model receives a picture taken at a garden by a member of the general public, and it is a bit blurry.  The model should still be able to successfully identify the flower at the same rate as non-blurry images. Test data needs to include blurred flower images.  Blurred images will be created using ImageMagick. Three datasets will be generated, each with different amounts of blur: minimal blur, maximum blur, and in between minimal and maximum blur. Blurry images are successfully identified at rates equal to that of non-blurred images. This will be measured using the Wilcoxon Rank-Sum test, with significance at p-value <=0.05.
* **Robustness - Model Robust to Noise (Channel Loss)**
  * The model receives a picture taken at a garden using a loaned device. These devices are known to sometimes lose a channel (i.e., RGB channel). The model should still be able to successfully identify the flower at the same rate as full images. Test data needs to include images with a missing channel. Test images will be generated by removing the R, G and B channels in the original test data using ImageMagic, therefore producing three data sets. Images with a missing channel are successfully identified at rates equal to that of original images. This will be measured using the Wilcoxon Rank-Sum test, with significance at p-value <=0.05.
* **Performance on Operational Platform**
  * The model will need to run on the devices loaned out by the garden centers to visitors. These are small, inexpensive devices with limited CPU power, as well as limited memory and disk space (512 MB and 128 GB, respectively). The original test dataset can be used. 1- Executing the model on the loaned platform will not exceed maximum CPU usage of 30% to ensure reasonable response time. CPU usage will be measure using ps. 2- Memory usage at inference time will not exceed available memory of 512 MB. This will be measured using pmap. 3 - Disk usage will not exceed available disk space of 128 GB. This will be measured using by adding the size of each file in the path for the model code.
* **Interpretability - Understanding Model Results**
  * The application that runs on the loaned device should indicate the main features that were used to recognize the flower, as part of the educational experience. The app will display the image highlighting the most informative features in flower identification, in addition to the flower name. The original test data set can be used. The model needs to return evidence, in this case a heat map implementing the Integrated Gradients algorithm, showing the pixels that were most informative in the classification decision. This evidence should be returned with each inference. 

  TODO: add new scenarios



# 1. Requirements

## 1.1 Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [18]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(
    store_path, exist_ok=True
)  # Ensure we are creating the folder if it is not there.

set_context("OxfordFlower", "0.0.1")
set_store(f"local://{store_path}")

## 1.2 Build a `NegotiationCard`

In MLTE, we negotiation requirements with the help of a `NegotiationCard`.

In [19]:
label_dict = {
    0: "_mexican_aster",
    1: "alpine_sea_holly",
    2: "anthurium",
    3: "artichoke",
    4: "arum_lily",
    5: "azalea",
    6: "ball_moss",
    7: "ballon_flower",
    8: "barberton_daisy",
    9: "bearded_iris",
    10: "bee_balm",
    11: "bird_of_paradise",
    12: "bishop_of_llandaf_dahlia",
    13: "black-eyed_susan",
    14: "blackberry_lily",
    15: "blanket_flower",
    16: "bolero_deep_blue",
    17: "bougainvillea",
    18: "bromelia",
    19: "buttercup",
    20: "california_poppy",
    21: "camellia",
    22: "canna_lily",
    23: "canterbury_bells",
    24: "cape_flower",
    25: "carnation",
    26: "cattleya",
    27: "cautleya_spicata",
    28: "clematis",
    29: "coltsfoot",
    30: "columbine",
    31: "common_dandelion",
    32: "corn_poppy",
    33: "cyclamen",
    34: "daffodil",
    35: "dahlia",
    36: "desert_rose",
    37: "english_marigold",
    38: "fire_lily",
    39: "foxglove",
    40: "frangipani",
    41: "fritillary",
    42: "garden_phlox",
    43: "gaura",
    44: "gazania",
    45: "geranium",
    46: "globe_flower",
    47: "globe_thistle",
    48: "grape_hyacinth",
    49: "great_masterwort",
    50: "hard-leaved_pocket_orchid",
    51: "hibiscus",
    52: "hippesatrum",
    53: "japanese_anemone",
    54: "king_protea",
    55: "lenten_rose",
    56: "lotus",
    57: "love_in_the_mist",
    58: "magnolia",
    59: "mallow",
    60: "marigold",
    61: "mexican_petunia",
    62: "monkshood",
    63: "moon_orchild",
    64: "morning_glory",
    65: "osteospermum",
    66: "oxeye_daisy",
    67: "passion_flower",
    68: "pelargonium",
    69: "peruvian_lily",
    70: "petunia",
    71: "pincushion_flower",
    72: "poinsettia",
    73: "primrose",
    74: "primula",
    75: "prince_of_whales_feather",
    76: "purple_coneflower",
    77: "red_ginger",
    78: "rose",
    79: "siam_tulip",
    80: "silverbush",
    81: "snapdragon",
    82: "spear_thistle",
    83: "spring_crocus",
    84: "stemless_gentain",
    85: "sunflower",
    86: "swear_pea",
    87: "sweet_william",
    88: "sword_lily",
    89: "thorn_apple",
    90: "tiger_lily",
    91: "tithonia_(incorrectly_labeled_as_orange_dahlia)",
    92: "toad_lily",
    93: "tree_mallow",
    94: "tree_poppy",
    95: "trumpet_creeper",
    96: "wallflower",
    97: "water_lily",
    98: "watercress",
    99: "wild_pansy",
    100: "windflower",
    101: "yellow_iris",
}

In [20]:
from mlte.negotiation.artifact import NegotiationCard
from mlte.model.shared import (
    MetricDescriptor,
    DataDescriptor,
    DataClassification,
    FieldDescriptor,
    LabelDescriptor,
    ModelDescriptor,
    ModelResourcesDescriptor,
    ModelIODescriptor,
    QASDescriptor,
    SystemDescriptor,
    GoalDescriptor,
    ProblemType,
    RiskDescriptor
)

card = NegotiationCard(
    system=SystemDescriptor(
        goals=[
            GoalDescriptor(
                description="The model should perform well.",
                metrics=[
                    MetricDescriptor(
                        description="accuracy",
                        baseline="Better than random chance.",
                    )
                ],
            )
        ],
        problem_type=ProblemType.CLASSIFICATION,
        task="Flower Classification",
        usage_context="A handheld flower identification device.",
        risks=RiskDescriptor(
            fp="The wrong type of flower is identified.",
            fn="The flower is not identified.",
            other="N/A",
        ),
    ),
    data=[
        DataDescriptor(
            description="Oxford flower dataset.",
            classification=DataClassification.UNCLASSIFIED,
            access="None",
            labeling_method="by hand",
            fields=[
                FieldDescriptor(
                    name="filename",
                    description="path to flower image.",
                    type="string to png file",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Kingdom",
                    description="The second highest taxonomic rank.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Phylum",
                    description="The taxonomic rank below kingdom and above Clade 1.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Clade1",
                    description="The taxonomic rank below Phylum and above Clade 2.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Clade2",
                    description="The taxonomic rank below Clade 1 and above Clade 3.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Clade3",
                    description="The taxonomic rank below Clade 2 and above Order.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Order",
                    description="The taxonomic rank below Clade 3 and above Family.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Family",
                    description="The taxonomic rank below Order and above Subfamily.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Subfamily",
                    description="The taxonomic rank below Family and above Genus.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Genus",
                    description="The taxonomic rank below Subfamily and above Species.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Common Name",
                    description="Image of flower including background.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Other Name",
                    description="Image of flower including background.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
                FieldDescriptor(
                    name="Label Name",
                    description="Image Label.",
                    type="string",
                    expected_values="N/A",
                    missing_values="N/A",
                    special_values="N/A",
                ),
            ],
            labels=[
                LabelDescriptor(description=label_dict[k], percent=0.0)
                for k in label_dict
            ],
            policies="N/A",
            rights="N/A",
            source="https://archive.ics.uci.edu/dataset/53/iris",
        )
    ],
    model=ModelDescriptor(
        development_compute_resources=ModelResourcesDescriptor(
            cpu="1", gpu="0", memory="6MiB", storage="2KiB"
        ),
        deployment_platform="local server",
        capability_deployment_mechanism="API",
        input_specification=[
            ModelIODescriptor(
                name="i1", description="description", type="Vector[150]"
            )
        ],
        output_specification=[
            ModelIODescriptor(
                name="o1", description="description", type="Vector[3]"
            )
        ],
        production_compute_resources=ModelResourcesDescriptor(
            cpu="1",
            gpu="0",
            memory="6MiB",
            storage="2KiB",
        ),
    ),
    qas=[
        QASDescriptor(
            quality="Fairness - Model Impartial to Photo Location",
            stimulus="The model receives a picture taken at the garden.",
            source="Provider of the new picture.",
            environment="Test data needs to include pictures of the flowers from the different gardens, grouped by the garden that the image was taken at. The quantity of the flower images should be representative of the garden population they are taken from.",
            response="Model can correctly identify the correct flowers.",
            measure="The total accuracy of the model across each garden population should be higher or equal to 0.9.",
        ),
        QASDescriptor(
            quality="Robustness to Noise (Image Blur)",
            stimulus="The model receives a picture taken at the garden which is a bit blurry.",
            source="By a member of the general public.",
            environment="Test data needs to include blurred flower images. Blurred images will be created using ImageMagick. Three datasets will be generated, each with different amounts of blur: minimal blur, maximum blur, and in between minimal and maximum blur.",
            response="The model should still be able to successfully identify the flower at the same rate as non-blurry images.",
            measure="This will be measured using the Wilcoxon Rank-Sum test, with significance at p-value <=0.05.",
        ),
        QASDescriptor(
            quality="Robustness to Noise (Channel Loss)",
            stimulus="The model receives a picture taken at the garden using a loaned device. These devices are known to sometimes lose a channel (i.e., RGB channel). ",
            source="By a member of the general public.",
            environment="Test data needs to include images with a missing channel. Test images will be generated by removing the R, G and B channels in the original test data using ImageMagic, therefore producing three data sets.",
            response="The model should still be able to successfully identify the flower at the same rate as full images.",
            measure="This will be measured using the Wilcoxon Rank-Sum test, with significance at p-value <=0.05.",
        ),
        QASDescriptor(
            quality="Performance on Operational Platform",
            stimulus="The model receives a picture taken at the garden using a loaned device. The model will need to run on this device.",
            source="By a member of the general public.",
            environment="The original test dataset can be used.",
            response="The model will not exceed the limited CPU power, as well as limited memory and disk space (512 MB and 128 GB, respectively) available.",
            measure=(
                "1- Executing the model on the loaned platform will not exceed maximum CPU usage of 30% to ensure reasonable response time. CPU usage will be measured using ps."
                + "2- Memory usage at inference time will not exceed available memory of 512 MB. This will be measured using pmap."
                + "3 - Disk usage will not exceed available disk space of 128 GB. This will be measured using by adding the size of each file in the path for the model code."
            ),
        ),
        QASDescriptor(
            quality="Interpretability - Understanding Model Results",
            stimulus="The model receives a picture taken at the garden using a loaned device.",
            source="By a member of the general public.",
            environment="A validation dataset with feature and label distributions broadly similar to the test dataset.",
            response="The app will display the image highlighting the most informative features in flower identification, in addition to the flower name. ",
            measure="Identify flowers correctly at least 90% of the time during normal operation.",
        ),
        # Start of table 2b
        QASDescriptor(
            quality="Functional Correctness: Accuracy",
            stimulus="The model receives receives a picture taken at the garden.",
            source="By a member of the general public.",
            environment="The original test dataset can be used.",
            response="Model can correctly identify the correct flowers.",
            measure="The model needs to return evidence, in this case a heat map implementing the Integrated Gradients algorithm, showing the pixels that were most informative in the classification decision. This evidence should be returned with each inference.",
        ),
        QASDescriptor(
            quality="Functional Correctness: Input and Output Specification",
            stimulus="The model receives receives a picture taken at the garden.",
            source="By a member of the general public.",
            environment="The original test dataset can be used.",
            response="During test execution all data in the test dataset produces an output that conforms to the output specification.",
            measure="The model needs to return evidence, in this case a heat map implementing the Integrated Gradients algorithm, showing the pixels that were most informative in the classification decision. This evidence should be returned with each inference.",
        ),
        QASDescriptor(
            quality="Resilience: Input Validation",
            stimulus="The model receives receives a picture taken at the garden.",
            source="Test dataset with invalid input formatting",
            environment="During normal operation",
            response="The ML pipeline will create a log entry with the tag 'Model - Input Validation Error - <Input>.'",
            measure="Error present in log",
        ),
        QASDescriptor(
            quality="Monitorability: Detect OOD inputs ",
            stimulus="The model receives receives a picture taken at the garden.",
            source="Input validation dataset designed to induce known failures, generated based on Equivalence Testing and Boundary Testing.",
            environment="During normal operation",
            response="Produce log entries for erroneous inputs.",
            measure="100% of errors produced by erroneous inputs are present in log files.",
        ),
        QASDescriptor(
            quality="Monitorability: Monitor shifts in output (confidence) distribution.",
            stimulus="The model receives receives a picture taken at the garden.",
            source="Test dataset with known shift in output confidence.",
            environment="During normal operation",
            response="Log entries are produced when distribution shift is detected.",
            measure="Confidence shift error is present in log files.",
        ),
        QASDescriptor(
            quality="Performance: Inference Time on Operational Platform",
            stimulus="The model receives receives a picture taken at the garden.",
            source="The origional test dataset can be used",
            environment="During normal operation",
            response="Log inference time during operation",
            measure="inference time should be less than 2 seconds.",
        ),
    ],
)
card.save(force=True, parents=True)

In [21]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(
    store_path, exist_ok=True
)  # Ensure we are creating the folder if it is not there.

set_context("OxfordFlower", "0.0.1")
set_store(f"local://{store_path}")

## 1.3 Define a Specification

In the first phase of SDMT, we define a `Specification` that represents the requirements the completed model must meet in order to be acceptable for use in the system into which it will be integrated.

In MLTE, we define requirements by constructing a specification (`Spec`). For each property, we define the validations to perform as well. Note that several new `Value` types (`MultipleAccuracy`, `RankSums`, `MultipleRanksums`) had to be created to define the validation methods that will validate each Condition.

In [ ]:
from mlte.spec.spec import Spec

# The Properties we want to validate, associated with our scenarios.
from mlte.property.costs.storage_cost import StorageCost
from mlte.property.fairness.fairness import Fairness
from mlte.property.robustness.robustness import Robustness
from mlte.property.interpretability.interpretability import Interpretability
from mlte.property.costs.predicting_memory_cost import PredictingMemoryCost
from mlte.property.costs.predicting_compute_cost import PredictingComputeCost

# The Value types we will use to validate each condition.
from mlte.measurement.storage import LocalObjectSize
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.memory import LocalProcessMemoryConsumption
from mlte.value.types.image import Image
from mlte.value.types.real import Real
from values.multiple_accuracy import MultipleAccuracy
from values.ranksums import RankSums
from values.multiple_ranksums import MultipleRanksums
from properties.monitorability import Monitorability
from properties.interoperability import Interoperability
from properties.accuracy import Accuracy
from values.string import String


# The full spec. Note that the Robustness Property contains conditions for both Robustness scenarios.
spec = Spec(
    properties={
        Fairness(
            "Important check if model performs well accross different populations"
        ): {
            "accuracy across gardens": MultipleAccuracy.all_accuracies_more_or_equal_than(
                0.9
            )
        },
        Robustness("Robust against blur and noise"): {
            "ranksums blur2x8": RankSums.p_value_greater_or_equal_to(0.05 / 3),
            "ranksums blur5x8": RankSums.p_value_greater_or_equal_to(0.05 / 3),
            "ranksums blur0x8": RankSums.p_value_greater_or_equal_to(0.05 / 3),
            "multiple ranksums for clade2": MultipleRanksums.all_p_values_greater_or_equal_than(
                0.05
            ),
            "multiple ranksums between clade2 and 3": MultipleRanksums.all_p_values_greater_or_equal_than(
                0.05
            ),
        },
        StorageCost("Critical since model will be in an embedded device"): {
            "model size": LocalObjectSize.value().less_than(3000)
        },
        PredictingMemoryCost(
            "Useful to evaluate resources needed when predicting"
        ): {
            "predicting memory": LocalProcessMemoryConsumption.value().average_consumption_less_than(
                512000.0
            )
        },
        PredictingComputeCost(
            "Useful to evaluate resources needed when predicting"
        ): {
            "predicting cpu": LocalProcessCPUUtilization.value().max_utilization_less_than(
                30.0
            )
        },
        Interpretability("Important to understand what the model is doing"): {
            "image attributions": Image.ignore("Inspect the image.")
        },
        # Functional Correctness: accuracy
        Accuracy("Measure the overall accuracy of your end to end pipeline"): {
            "overall accuracy": Real.greater_than(0.9)
        },
        # Functional Correctness: I/O spec
        Interoperability(
            "Model output format must conform to specified format"
        ): {
            "output format validation": String.contains(
                "Model - Output Validation Error"
            )
        },
        # Resilience: Input Validation
        Resilience("Model inputs must conform to specified format"): {
            "input format validation": String.contains(
                "Model - Input Validation Error"
            )
        },
        # Monitorability: Detect OOD inputs
        Monitorability("Monitor inputs for OOD data"): {
            "detect ood inputs": String.contains("Model - Input OOD Error")
        },
        # Monitorability: Detect output shifts
        Monitorability("Monitor outputs for unexpected shifts"): {
            "monitor output confidence shift": String.contains(
                "Model - Output Confidence Error"
            )
        },
        # Performance: Inference time on Operational Platform
        PredictingComputeCost(
            "End to end latency for inference on operational platform"
        ): {"predicting cpu time": Real.less_than(2.0)},
    }
)
spec.save(parents=True, force=True)